In [7]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
import time
import requests
from PIL import Image
from io import BytesIO
import os

# Update with path to your msedgedriver
service = Service('C:/edgedriver_win64/msedgedriver.exe')
wd = webdriver.Edge(service=service)

query = input("Enter the search query: ")
url = f"https://www.google.com/search?q={query}&tbm=isch"
wd.get(url)

# Pause to allow thumbnails to load
time.sleep(5)

# Ask user for the number of images to scrape
num_images = int(input("Enter the number of images you want to download: "))
print('----------------------------------------------------')

# Find all thumbnail images
thumbnails = wd.find_elements(By.CLASS_NAME, "mNsIhb")

# Validate the number of thumbnails
if len(thumbnails) < num_images:
    print(f"Only {len(thumbnails)} thumbnails found. Adjusting number of images to download.")
    num_images = len(thumbnails)

# Ensure the 'pics' folder exists
folder_path = 'pics'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Download images
for i in range(num_images):
    try:
        # Click the thumbnail to open the full-size image
        thumbnail = thumbnails[i]
        wd.execute_script("arguments[0].scrollIntoView();", thumbnail)
        wd.execute_script("arguments[0].click();", thumbnail)

        # Pause to allow the larger image to load
        time.sleep(5)

        # Find the larger image and get its URL (update this selector as needed)
        large_image = wd.find_element(By.CSS_SELECTOR, "img.sFlh5c.FyHeAf.iPVvYb")
        image_src = large_image.get_attribute("src")

        # Download the image
        response = requests.get(image_src)
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))
            # Save image to 'pics' folder
            image.save(f'{folder_path}/{query}_{i + 1}.jpg')
            print(f"Image {i + 1} successfully downloaded and saved as '{folder_path}/{query}_{i + 1}.jpg'")
        else:
            print(f"Failed to retrieve image {i + 1}. Status code: {response.status_code}")

        # Update the thumbnails list after downloading
        thumbnails = wd.find_elements(By.CLASS_NAME, "mNsIhb")

    except Exception as e:
        # print(f"An error occurred with image {i + 1}: {e}")  # Uncomment if want to print the error
        print(f"An error occurred with image {i + 1}:")
        continue  # Move on to the next image


Enter the search query:  pen
Enter the number of images you want to download:  5


----------------------------------------------------
Image 1 successfully downloaded and saved as 'pics/pen_1.jpg'
Image 2 successfully downloaded and saved as 'pics/pen_2.jpg'
Image 3 successfully downloaded and saved as 'pics/pen_3.jpg'
Image 4 successfully downloaded and saved as 'pics/pen_4.jpg'
An error occurred with image 5:
